In [ ]:
# Mount Google Drive to access files (if needed)
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import nltk
import re
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, GlobalMaxPooling1D, Dropout, Input, Attention, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

# Check if GPU is available and use it
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print(f'Found GPU at: {device_name}')

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Define a function for text preprocessing (stemming and cleaning)
def preprocess_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    ps = PorterStemmer()
    text = [ps.stem(word) for word in text if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    return text

# Load dataset (adjust path as per your file location)
news_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/fakeNewsData.csv')
news_dataset = news_dataset.fillna('')

# Combine author and title into content
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']
news_dataset['content'] = news_dataset['content'].apply(preprocess_text)

# Split dataset into train and test sets
X = news_dataset['content']
y = news_dataset['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorization using CountVectorizer
cv = CountVectorizer(max_features=5000)
X_train_cv = cv.fit_transform(X_train).toarray()
X_test_cv = cv.transform(X_test).toarray()

# Vectorization using TF-IDF
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

# Vectorization using Word2Vec
sentences = [text.split() for text in X_train]
wv_model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=2, workers=4)

# Prepare Word2Vec vectors for training
def get_word2vec_embeddings(wv_model, texts, max_len):
    embeddings = np.zeros((len(texts), max_len, wv_model.vector_size))
    for i, text in enumerate(texts):
        words = text.split()
        for j, word in enumerate(words):
            if j == max_len:
                break
            if word in wv_model.wv:
                embeddings[i, j] = wv_model.wv[word]
    return embeddings

max_len = 100  # Adjusted max length to manage memory usage
X_train_word2vec = get_word2vec_embeddings(wv_model, X_train, max_len)
X_test_word2vec = get_word2vec_embeddings(wv_model, X_test, max_len)

# Combine CountVectorizer, TF-IDF, and Word2Vec embeddings into a single feature set
X_train_hybrid = np.concatenate((X_train_cv, X_train_tfidf, X_train_word2vec.reshape((X_train_word2vec.shape[0], -1))), axis=1)
X_test_hybrid = np.concatenate((X_test_cv, X_test_tfidf, X_test_word2vec.reshape((X_test_word2vec.shape[0], -1))), axis=1)

# Define CNN with Attention (step 5)
input_layer = Input(shape=(X_train_hybrid.shape[1],))
dense_layer = Dense(128, activation='relu')(input_layer)
dense_layer = Dense(64, activation='relu')(dense_layer)

conv_layer = Conv1D(filters=128, kernel_size=5, activation='relu')(tf.expand_dims(dense_layer, -1))
attention_data = Conv1D(filters=128, kernel_size=3, activation='relu')(tf.expand_dims(dense_layer, -1))
attention_layer = Attention()([conv_layer, attention_data])
flatten_layer = GlobalMaxPooling1D()(attention_layer)
dropout_layer = Dropout(0.5)(flatten_layer)
output_layer = Dense(1, activation='sigmoid')(dropout_layer)

model_attention = Model(inputs=input_layer, outputs=output_layer)

model_attention.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

print("\nTraining CNN with Attention and Hybrid Features...")
history_attention = model_attention.fit(X_train_hybrid, y_train, epochs=20, batch_size=32, validation_data=(X_test_hybrid, y_test), verbose=1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found GPU at: /device:GPU:0


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Training CNN with Attention and Hybrid Features...
Epoch 1/20
520/520 [==============================] - 11s 8ms/step - loss: 0.2742 - accuracy: 0.9123 - val_loss: 0.0880 - val_accuracy: 0.9796
Epoch 2/20
520/520 [==============================] - 3s 6ms/step - loss: 0.0558 - accuracy: 0.9873 - val_loss: 0.0497 - val_accuracy: 0.9863
Epoch 3/20
520/520 [==============================] - 4s 7ms/step - loss: 0.0256 - accuracy: 0.9933 - val_loss: 0.0467 - val_accuracy: 0.9887
Epoch 4/20
520/520 [==============================] - 3s 6ms/step - loss: 0.0165 - accuracy: 0.9953 - val_loss: 0.0515 - val_accuracy: 0.9865
Epoch 5/20
520/520 [==============================] - 3s 6ms/step - loss: 0.0103 - accuracy: 0.9975 - val_loss: 0.0469 - val_accuracy: 0.9887
Epoch 6/20
520/520 [==============================] - 3s 6ms/step - loss: 0.0053 - accuracy: 0.9987 - val_loss: 0.0398 - val_accuracy: 0.9916
Epoch 7/20
520/520 [==============================] - 4s 7ms/step - loss: 0.0099 - accuracy: 0.

In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score, recall_score, f1_score, matthews_corrcoef

# Accuracy
loss_attention, accuracy_attention = model_attention.evaluate(X_test_hybrid, y_test)

print(f"\nAttention CNN Test Accuracy: {accuracy_attention}")
print(f"\n Loss Attention: {loss_attention}")

# Predict probabilities for test set
y_pred_prob = model_attention.predict(X_test_hybrid)

# Convert probabilities to binary predictions (0 or 1)
y_pred = (y_pred_prob > 0.5).astype(int)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print(f"Confusion Matrix:\n{cm}")

# AUC Score
auc_score = roc_auc_score(y_test, y_pred_prob)
print(f"AUC Score: {auc_score:.4f}")

# Calculate True Positives, True Negatives, False Positives, False Negatives
tn, fp, fn, tp = cm.ravel()

# Sensitivity (Recall)
sensitivity = tp / (tp + fn)
print(f"Sensitivity (Recall): {sensitivity:.4f}")

# Specificity
specificity = tn / (tn + fp)
print(f"Specificity: {specificity:.4f}")

# Precision
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision:.4f}")

# F1 Score
f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1:.4f}")

# Matthews Correlation Coefficient (MCC)
mcc = matthews_corrcoef(y_test, y_pred)
print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")

130/130 [==============================] - 1s 4ms/step - loss: 0.0393 - accuracy: 0.9909

Attention CNN Test Accuracy: 0.9908654093742371

 Loss Attention: 0.039295632392168045
130/130 [==============================] - 1s 4ms/step
Confusion Matrix:
[[2106   26]
 [  12 2016]]
AUC Score: 0.9990
Sensitivity (Recall): 0.9941
Specificity: 0.9878
Precision: 0.9873
F1 Score: 0.9907
Matthews Correlation Coefficient (MCC): 0.9817
